In [70]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from date import Date

# '2015-10-10T13:00:00Z'
df = pd.read_json("activities2.json", dtype=True)
print(df.shape)
df.loc[:,'start_date'] = df['start_date'].map(Date)
rides = df.loc[df.type == 'Ride']
rides.loc[:,'weekNumber'] = rides['start_date'].apply(lambda x: x.weekNumber)
rides.loc[:,'month']= rides['start_date'].apply(lambda x: x.month)
rides.loc[:,'year'] = rides['start_date'].apply(lambda x: x.year)

(778, 58)


/Users/alexwurts/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/alexwurts/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [71]:
df[df.start_date.apply(lambda x: x.year == 2020)].start_date

748    2020 5 16 23
749    2020 5 15 23
750    2020 5 14 23
751     2020 5 7 22
752     2020 5 6 22
753     2020 5 4 22
754    2020 4 31 21
755    2020 4 28 21
756    2020 4 25 20
757    2020 4 22 20
758    2020 4 21 20
759    2020 4 19 19
760    2020 4 18 19
761    2020 4 17 19
762     2020 4 7 18
763     2020 4 4 17
764     2020 4 4 17
765     2020 4 3 17
766      2020 2 3 8
Name: start_date, dtype: object

In [72]:

dataForVis = rides[['weekNumber', 'month', 'year', 'start_date', 'distance', 'total_elevation_gain', 'elapsed_time', 'pr_count', 'average_speed', 'average_watts', 'average_temp']]
count = pd.Series(1, index=dataForVis.index)
dataForVis.loc[:,'count'] = count

dataForVis.loc[:,'distance'] = dataForVis['distance'] / 1609
dataForVis.loc[:,'total_elevation_gain'] = dataForVis['total_elevation_gain'] * 3.28083
dataForVis.loc[:,'average_speed'] = dataForVis['average_speed'] * 2.23694
dataForVis.loc[:,'average_temp'] = (dataForVis['average_temp'] * (9/5)) + 32


In [73]:
# rides.hist(column=rides.select_dtypes(include=['float64']).columns)
numberCols = rides.select_dtypes(include=['float64'])
numberCols = numberCols.drop(columns=['elev_high', 'elev_low', 'from_accepted_tag', 'start_latitude', 'start_longitude', 'workout_type', 'device_watts'])


In [74]:
grouping = dataForVis.groupby(['year', 'month'])

rides_by_month = grouping.mean()
totals = grouping.sum()[['pr_count', 'distance', 'total_elevation_gain', 'elapsed_time', 'count']]
rides_by_month[['pr_count', 'distance', 'total_elevation_gain', 'elapsed_time', 'count']] = totals

## Filling in missing data using new index
monthNumbers = list(range(0,12))
years = rides['year'].unique()
newIndexList = [(y, m) for m in monthNumbers for y in years]
new_index = pd.MultiIndex.from_tuples(newIndexList, names=['year', 'month'])
rides_by_month = rides_by_month.reindex(new_index)
rides_by_month.fillna(0, inplace=True)

rides_by_month.reset_index(inplace=True)
rides_by_month.to_csv('../docs/rides_by_month.csv')


In [75]:
grouping = dataForVis.groupby(['year', 'weekNumber'])

rides_by_week = grouping.mean()
totals = grouping.sum()[['pr_count', 'distance', 'total_elevation_gain', 'elapsed_time', 'count']]
rides_by_week[['pr_count', 'distance', 'total_elevation_gain', 'elapsed_time', 'count']] = totals

## Filling in missing data using new index
weekNumbers = list(range(0,52))
years = rides['year'].unique()
newIndexList = [(y, w) for w in weekNumbers for y in years]
new_index = pd.MultiIndex.from_tuples(newIndexList, names=['year', 'weekNumber'])
rides_by_week = rides_by_week.reindex(new_index)
rides_by_week.fillna(0, inplace=True)
rides_by_week.reset_index(inplace=True)
rides_by_week.to_csv('../docs/rides_by_week.csv')

In [76]:
selected = rides[['average_heartrate', 'average_speed', 'weighted_average_watts', 'distance', 'kilojoules', 'max_watts']]

In [77]:
## Save to Json

rides_by_week.to_json("../docs/rides_by_week.json", orient='records')
rides_by_month.to_json("../docs/rides_by_month.json",orient='records')

In [78]:
selectedAttributes = rides[['distance', 'total_elevation_gain', 'average_watts', 'average_speed',
                            'start_date', 'average_temp', 'average_cadence', 'moving_time', 'pr_count',
                           'month', 'year', 'weekNumber', 'id']]

In [79]:
rides.columns

Index(['resource_state', 'athlete', 'name', 'distance', 'moving_time',
       'elapsed_time', 'total_elevation_gain', 'type', 'workout_type', 'id',
       'external_id', 'upload_id', 'start_date', 'start_date_local',
       'timezone', 'utc_offset', 'start_latlng', 'end_latlng', 'location_city',
       'location_state', 'location_country', 'start_latitude',
       'start_longitude', 'achievement_count', 'kudos_count', 'comment_count',
       'athlete_count', 'photo_count', 'map', 'trainer', 'commute', 'manual',
       'private', 'visibility', 'flagged', 'gear_id', 'from_accepted_tag',
       'upload_id_str', 'average_speed', 'max_speed', 'average_cadence',
       'has_heartrate', 'average_heartrate', 'max_heartrate',
       'heartrate_opt_out', 'display_hide_heartrate_option', 'elev_high',
       'elev_low', 'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score', 'device_watts', 'average_watts', 'kilojoules',
       'average_temp', 'weighted_average_watts', 'max_watts', '

In [80]:
selectedAttributes.to_json('rides.json', orient='records')

In [81]:
selectedAttributes

,distance,total_elevation_gain,average_watts,average_speed,start_date,average_temp,average_cadence,moving_time,pr_count,month,year,weekNumber,id
7,13518.5,0.0,NaN,7.510,2015 9 10 40,NaN,NaN,1800,0,9,2015,40,410281414
9,12391.9,0.0,NaN,6.884,2015 9 9 39,NaN,NaN,1800,0,9,2015,39,410904887
10,18829.3,0.0,NaN,6.276,2015 9 8 39,NaN,NaN,3000,0,9,2015,39,409111739
12,16898.1,0.0,NaN,7.041,2015 9 6 39,NaN,NaN,2400,0,9,2015,39,407717016
45,5310.9,0.0,NaN,6.960,2015 9 26 42,NaN,NaN,763,0,9,2015,42,420887313
...,...,...,...,...,...,...,...,...,...,...,...,...,...
771,45891.8,416.0,156.5,7.891,2019 8 9 35,20.0,NaN,5816,5,8,2019,35,2694266090
774,19000.4,100.0,472.0,8.586,2019 8 5 35,19.0,92.1,2213,1,8,2019,35,2683085824
775,33019.4,162.0,437.5,8.001,2019 8 4 35,24.0,89.8,4127,0,8,2019,35,2680219375
776,12011.0,65.0,464.0,7.897,2019 8 3 35,19.0,90.8,1521,3,8,2019,35,2677619398
